In [2]:
## K-Center Coreset
## Karan Vombatkere, Dec 2021

import pandas as pd
import numpy as np
import helper_functions as hf

In [3]:
class Coreset_kCenter:
    """
    Class to compute K-Center Coreset
    Parameters
    ----------
        x_arr : numpy array data from Coreset_Util
        k_val : Value of k for computing k-center Coreset
        epsilon : epsilon value for Coreset
    """
    
    #Initialize with parameters
    def __init__(self, x_arr, k_val, epsilon):
        self.x_array = x_arr
        self.k = k_val
        self.epsilon = epsilon
        

    #Compute Greedy k-center with polynomial 2-approximation
    def greedy_kcenter(self):
        self.centers = []

        #Randomly select a point and add to centers
        randomPointIndex = np.random.randint(0,len(self.x_array)+1)
        s_1 = self.x_array[randomPointIndex]
        self.centers.append(s_1)

        while len(self.centers) < self.k:
            maximum_dist_point = hf.max_distance(self.x_array, self.centers)
            self.centers.append(maximum_dist_point)

        return self.centers

    